# Proceso ETL: Una Guía Práctica con Julia

## 📚 Objetivos de Aprendizaje

Al completar este notebook, los estudiantes serán capaces de:
- Comprender los conceptos fundamentales de ETL (Extract, Transform, Load)
- Implementar cada fase del proceso ETL usando Julia y DataFrames.jl
- Aplicar técnicas de limpieza y transformación de datos
- Manejar errores comunes en pipelines de datos
- Diseñar pipelines ETL escalables y mantenibles

## 🎯 ¿Qué es ETL?

**ETL** es un proceso fundamental en ingeniería de datos que permite:
- **Extract (Extraer)**: Obtener datos de múltiples fuentes
- **Transform (Transformar)**: Limpiar, validar y estructurar los datos
- **Load (Cargar)**: Almacenar los datos procesados en el destino final

Este notebook es una guía completa que cubre desde conceptos básicos hasta temas avanzados, con ejemplos prácticos y ejercicios para trabajar con pipelines ETL en entornos reales de ingeniería de datos usando **Julia**.

## El Pipeline ETL de Referencia

Nos basaremos en la siguiente estructura de pipeline, que representa un flujo ETL clásico:

In [1]:
function etl_pipeline()
    # 1. Extraer datos desde una fuente original
    raw_data = extract_from_database()
    
    # 2. Transformar los datos aplicando reglas y limpiando
    cleaned_data = apply_business_rules(raw_data)
    
    # 3. Normalizar el esquema para que sea consistente
    structured_data = normalize_schema(cleaned_data)
    
    # 4. Cargar los datos transformados a su destino final
    load_to_warehouse(structured_data)
end

A continuación, implementaremos cada una de estas funciones con ejemplos claros.

## 📦 Librerías y Configuración Inicial

Para nuestros ejemplos, usaremos las siguientes librerías esenciales en ingeniería de datos con Julia:

In [10]:
# Instalar paquetes necesarios (ejecutar solo la primera vez)
using Pkg
# Pkg.add(["DataFrames", "CSV", "Dates", "JSON3", "Logging", "Statistics"])

# Cargar librerías
using DataFrames        # Manipulación y análisis de datos
using CSV              # Lectura y escritura de archivos CSV
using Dates            # Manejo de fechas y tiempos
using JSON3            # Manejo de datos JSON
using Logging          # Sistema de logs
using Statistics       # Funciones estadísticas básicas

# Configuración de logging para monitorear el pipeline
logger = ConsoleLogger(stdout, Logging.Info)
global_logger(logger)

println("✅ Librerías cargadas exitosamente")

✅ Librerías cargadas exitosamente


## 🧠 Conceptos Fundamentales de ETL

### Tipos de Procesamiento ETL

1. **Batch Processing (Procesamiento por lotes)**
   - Procesa grandes volúmenes de datos en intervalos programados
   - Ideal para reportes diarios, semanales o mensuales
   - Ejemplo: Procesar todas las transacciones del día anterior

2. **Real-time Processing (Procesamiento en tiempo real)**
   - Procesa datos tan pronto como llegan
   - Crítico para aplicaciones que requieren respuesta inmediata
   - Ejemplo: Detección de fraude en transacciones

3. **Near Real-time Processing (Procesamiento casi en tiempo real)**
   - Procesa datos con un pequeño retraso (segundos o minutos)
   - Balance entre velocidad y eficiencia de recursos
   - Ejemplo: Actualización de dashboards cada 5 minutos

### Calidad de Datos - Las 6 Dimensiones

1. **Exactitud**: Los datos reflejan la realidad
2. **Completitud**: No hay valores faltantes críticos
3. **Consistencia**: Los datos son coherentes entre sistemas
4. **Validez**: Los datos cumplen con reglas de negocio
5. **Unicidad**: No hay duplicados no deseados
6. **Actualidad**: Los datos están actualizados

## 1. Extract: Extracción de Datos

### 🎯 Objetivos de la Fase Extract
- Conectar con múltiples fuentes de datos
- Extraer datos de manera eficiente
- Manejar diferentes formatos y protocolos
- Implementar estrategias de extracción incremental

### 📊 Fuentes de Datos Comunes
- **Bases de datos relacionales**: MySQL, PostgreSQL, SQL Server
- **Bases de datos NoSQL**: MongoDB, Cassandra, Redis
- **Archivos**: CSV, JSON, XML, Parquet
- **APIs REST**: Servicios web y microservicios
- **Sistemas de streaming**: Kafka, Kinesis

**Ejemplo:** Vamos a simular la extracción de datos de una base de datos SQL creando un DataFrame con datos de ejemplo.

In [11]:
function extract_from_database()
    """
    Simula la extracción de datos desde una base de datos.
    En un entorno real, esto se conectaría a una base de datos real.
    
    Returns:
        DataFrame: Datos crudos extraídos
    """
    try
        @info "Iniciando extracción de datos..."
        
        # Simulamos datos con problemas típicos de calidad
        data = DataFrame(
            ID_USER = [101, 102, 103, 104, 105, 106],
            user_name = ["Ana", "Luis", "Marta", "Juan", "Eva", missing],  # Valor nulo
            registration_date = ["2025-01-15", "2025-02-20", "2025-03-01", "2025-04-10", "2025-05-19", "2025-06-30"],
            total_spent = [150.5, 80.0, -999.0, 200.0, 45.25, "invalid"],  # Valor inválido
            country_code = ["ES", "MX", "ES", "CO", "es", "ES"],  # Inconsistencia en mayúsculas
            email = ["ana@email.com", "luis@email.com", "marta@email.com", "juan@email.com", "eva@email.com", "pedro@email.com"]
        )
        
        println("--- 1. Datos Crudos Extraídos ---")
        println("Registros extraídos: ", nrow(data))
        println("Columnas: ", names(data))
        println("\nPrimeras filas:")
        println(data)
        println("\nInformación del DataFrame:")
        println("Tipos de datos: ", eltype.(eachcol(data)))
        println("")
        
        @info "Extracción completada: $(nrow(data)) registros"
        return data
        
    catch e
        @error "Error en la extracción: $e"
        rethrow(e)
    end
end

extract_from_database (generic function with 1 method)

### ▶️ Ejecutemos la Extracción

Ahora vamos a ejecutar la función de extracción para ver los datos:

In [12]:
# Ejecutar la función de extracción
raw_data = extract_from_database()

[ Info: Iniciando extracción de datos...
--- 1. Datos Crudos Extraídos ---
Registros extraídos: 6
Columnas: ["ID_USER", "user_name", "registration_date", "total_spent", "country_code", "email"]

Primeras filas:
6×6 DataFrame
 Row │ ID_USER  user_name  registration_date  total_spent  country_code  email           
     │ Int64    String?    String             Any          String        String          
─────┼───────────────────────────────────────────────────────────────────────────────────
   1 │     101  Ana        2025-01-15         150.5        ES            ana@email.com
   2 │     102  Luis       2025-02-20         80.0         MX            luis@email.com
   3 │     103  Marta      2025-03-01         -999.0       ES            marta@email.com
   4 │     104  Juan       2025-04-10         200.0        CO            juan@email.com
   5 │     105  Eva        2025-05-19         45.25        es            eva@email.com
   6 │     106  missing    2025-06-30         invalid      ES     

Row,ID_USER,user_name,registration_date,total_spent,country_code,email
,Int64,String?,String,Any,String,String
1,101,Ana,2025-01-15,150.5,ES,ana@email.com
2,102,Luis,2025-02-20,80.0,MX,luis@email.com
3,103,Marta,2025-03-01,-999.0,ES,marta@email.com
4,104,Juan,2025-04-10,200.0,CO,juan@email.com
5,105,Eva,2025-05-19,45.25,es,eva@email.com
6,106,missing,2025-06-30,invalid,ES,pedro@email.com


### 📁 Ejemplo: Extracción desde Archivo CSV

Veamos cómo extraer datos desde un archivo CSV (otra fuente común):

In [13]:
function extract_from_csv(file_path="sample_products.csv")
    """
    Extrae datos desde un archivo CSV.
    
    Args:
        file_path (String): Ruta del archivo CSV
    
    Returns:
        DataFrame: Datos extraídos del CSV
    """
    try
        # Crear un archivo CSV de ejemplo
        sample_data = DataFrame(
            product_id = [1, 2, 3, 4, 5],
            product_name = ["Laptop", "Mouse", "Keyboard", "Monitor", "Headphones"],
            price = [999.99, 25.50, 75.00, 299.99, 89.99],
            category = ["Electronics", "Accessories", "Accessories", "Electronics", "Accessories"]
        )
        
        CSV.write(file_path, sample_data)
        
        # Leer el archivo CSV
        df = CSV.read(file_path, DataFrame)
        
        println("--- Extracción desde CSV ---")
        println(df)
        return df
        
    catch e
        if isa(e, SystemError)
            @error "Archivo no encontrado: $file_path"
            return DataFrame()
        else
            @error "Error leyendo CSV: $e"
            return DataFrame()
        end
    end
end

# Ejemplo de uso
products_df = extract_from_csv()

--- Extracción desde CSV ---
5×4 DataFrame
 Row │ product_id  product_name  price    category    
     │ Int64       String15      Float64  String15    
─────┼────────────────────────────────────────────────
   1 │          1  Laptop         999.99  Electronics
   2 │          2  Mouse           25.5   Accessories
   3 │          3  Keyboard        75.0   Accessories
   4 │          4  Monitor        299.99  Electronics
   5 │          5  Headphones      89.99  Accessories


Row,product_id,product_name,price,category
,Int64,String15,Float64,String15
1,1,Laptop,999.99,Electronics
2,2,Mouse,25.5,Accessories
3,3,Keyboard,75.0,Accessories
4,4,Monitor,299.99,Electronics
5,5,Headphones,89.99,Accessories


## 2. Transform: Aplicación de Reglas de Negocio

### 🎯 Objetivos de la Fase Transform
- Limpiar datos inconsistentes o erróneos
- Validar que los datos cumplan reglas de negocio
- Enriquecer datos con información adicional
- Aplicar transformaciones matemáticas o lógicas
- Normalizar formatos y estructuras

### 🔧 Técnicas Comunes de Transformación
- **Limpieza**: Eliminar duplicados, corregir errores tipográficos
- **Validación**: Verificar rangos, formatos, tipos de datos
- **Enriquecimiento**: Agregar datos calculados o de referencia
- **Normalización**: Estandarizar formatos y escalas
- **Agregación**: Resumir datos por grupos o períodos

**Ejemplo:** Aplicaremos las siguientes reglas:
1. Corregir valores erróneos: El valor `-999` en `total_spent` debe ser 0
2. Corregir inconsistencias: El código de país `es` debe ser `ES`
3. Filtrar datos: Solo usuarios con gasto mayor a 50
4. Manejar valores nulos en nombres de usuario

In [14]:
function apply_business_rules(raw_data::DataFrame)
    """
    Aplica reglas de negocio y limpieza de datos.
    
    Args:
        raw_data (DataFrame): Datos crudos a transformar
    
    Returns:
        DataFrame: Datos limpios y transformados
    """
    try
        @info "Iniciando transformación de datos..."
        cleaned_data = copy(raw_data)
        
        println("--- Análisis de Calidad de Datos ---")
        println("Registros iniciales: ", nrow(cleaned_data))
        println("Valores nulos por columna:")
        for col in names(cleaned_data)
            null_count = sum(ismissing.(cleaned_data[!, col]))
            println("  $col: $null_count")
        end
        println()
        
        # 1. Manejar valores nulos en user_name
        null_names = sum(ismissing.(cleaned_data.user_name))
        if null_names > 0
            println("⚠️  Encontrados $null_names nombres de usuario nulos")
            cleaned_data.user_name = coalesce.(cleaned_data.user_name, "Usuario_Desconocido")
        end
        
        # 2. Corregir valores erróneos en total_spent
        # Convertir valores no numéricos a missing, luego reemplazar -999 por 0
        cleaned_data.total_spent = [isa(val, Number) ? val : missing for val in cleaned_data.total_spent]
        invalid_spent = sum(ismissing.(cleaned_data.total_spent))
        if invalid_spent > 0
            println("⚠️  Encontrados $invalid_spent valores inválidos en total_spent")
            cleaned_data.total_spent = coalesce.(cleaned_data.total_spent, 0.0)
        end
        
        # Reemplazar -999 por 0
        cleaned_data.total_spent = [val == -999.0 ? 0.0 : val for val in cleaned_data.total_spent]
        
        # 3. Normalizar códigos de país
        cleaned_data.country_code = uppercase.(string.(cleaned_data.country_code))
        
        # 4. Aplicar regla de negocio: solo usuarios con gasto > 50
        initial_count = nrow(cleaned_data)
        cleaned_data = filter(row -> row.total_spent > 50, cleaned_data)
        filtered_count = initial_count - nrow(cleaned_data)
        
        println("📊 Registros filtrados (gasto <= 50): $filtered_count")
        println("📊 Registros finales: ", nrow(cleaned_data))
        
        println("\n--- 2. Datos Limpios (Reglas Aplicadas) ---")
        println(cleaned_data)
        println("")
        
        @info "Transformación completada: $(nrow(cleaned_data)) registros válidos"
        return cleaned_data
        
    catch e
        @error "Error en la transformación: $e"
        rethrow(e)
    end
end

apply_business_rules (generic function with 1 method)

### ▶️ Ejecutemos la Transformación

Ahora vamos a aplicar las reglas de negocio a nuestros datos extraídos:

In [15]:
# Ejecutar la transformación (necesitamos los datos de la extracción)
if @isdefined(raw_data)
    cleaned_data = apply_business_rules(raw_data)
else
    println("⚠️ Primero ejecuta la celda de extracción de datos")
end

[ Info: Iniciando transformación de datos...
--- Análisis de Calidad de Datos ---
Registros iniciales: 6
Valores nulos por columna:
  ID_USER: 0
  user_name: 1
  registration_date: 0
  total_spent: 0
  country_code: 0
  email: 0

⚠️  Encontrados 1 nombres de usuario nulos
⚠️  Encontrados 1 valores inválidos en total_spent
📊 Registros filtrados (gasto <= 50): 3
📊 Registros finales: 3

--- 2. Datos Limpios (Reglas Aplicadas) ---
3×6 DataFrame
 Row │ ID_USER  user_name  registration_date  total_spent  country_code  email          
     │ Int64    String     String             Float64      String        String         
─────┼──────────────────────────────────────────────────────────────────────────────────
   1 │     101  Ana        2025-01-15               150.5  ES            ana@email.com
   2 │     102  Luis       2025-02-20                80.0  MX            luis@email.com
   3 │     104  Juan       2025-04-10               200.0  CO            juan@email.com

[ Info: Transformación c

Row,ID_USER,user_name,registration_date,total_spent,country_code,email
,Int64,String,String,Float64,String,String
1,101,Ana,2025-01-15,150.5,ES,ana@email.com
2,102,Luis,2025-02-20,80.0,MX,luis@email.com
3,104,Juan,2025-04-10,200.0,CO,juan@email.com


### 🔍 Técnicas Avanzadas de Transformación

Veamos algunas técnicas adicionales de transformación:

In [16]:
function advanced_transformations(df::DataFrame)
    """
    Aplica transformaciones avanzadas a los datos.
    
    Args:
        df (DataFrame): DataFrame a transformar
    
    Returns:
        DataFrame: DataFrame con transformaciones aplicadas
    """
    df_transformed = copy(df)
    
    # 1. Crear columnas derivadas
    df_transformed.registration_year = [parse(Int, split(date, "-")[1]) for date in df_transformed.registration_date]
    df_transformed.registration_month = [parse(Int, split(date, "-")[2]) for date in df_transformed.registration_date]
    
    # 2. Categorizar gastos
    function categorize_spending(amount)
        if amount < 100
            return "Bajo"
        elseif amount < 200
            return "Medio"
        else
            return "Alto"
        end
    end
    
    df_transformed.spending_category = [categorize_spending(amount) for amount in df_transformed.total_spent]
    
    # 3. Validar emails (ejemplo básico)
    if "email" in names(df_transformed)
        df_transformed.email_valid = [occursin("@", email) for email in df_transformed.email]
    end
    
    println("--- Transformaciones Avanzadas ---")
    println(select(df_transformed, [:user_name, :total_spent, :spending_category, :registration_year]))
    
    return df_transformed
end

# Esta función se puede usar después de apply_business_rules
# df_advanced = advanced_transformations(cleaned_data)

advanced_transformations (generic function with 1 method)

## 3. Transform: Normalización del Esquema

### 🎯 Objetivos de la Normalización
- Estandarizar nombres de columnas
- Asegurar tipos de datos correctos
- Mantener consistencia entre sistemas
- Facilitar la integración con el destino

El objetivo es asegurar que el esquema de los datos (nombres de columnas, tipos de datos) sea consistente.

**Ejemplo:** Vamos a:
1. Cambiar nombres de columnas a un formato estándar (snake_case)
2. Asegurar que `registration_date` sea de tipo fecha
3. Reordenar columnas según el esquema del destino

In [17]:
function normalize_schema(cleaned_data::DataFrame)
    """
    Normaliza el esquema de datos según estándares del data warehouse.
    
    Args:
        cleaned_data (DataFrame): Datos limpios a normalizar
    
    Returns:
        DataFrame: Datos con esquema normalizado
    """
    try
        @info "Iniciando normalización del esquema..."
        structured_data = copy(cleaned_data)
        
        println("--- Análisis del Esquema Original ---")
        println("Columnas originales: ", names(structured_data))
        println("Tipos de datos originales:")
        for (name, type) in zip(names(structured_data), eltype.(eachcol(structured_data)))
            println("  $name: $type")
        end
        println()
        
        # 1. Normalizar nombres de columnas (snake_case)
        column_mapping = Dict(
            :ID_USER => :user_id,
            :user_name => :user_name,
            :registration_date => :registration_date,
            :total_spent => :total_spent,
            :country_code => :country_code,
            :email => :email_address
        )
        
        # Solo renombrar columnas que existen
        existing_columns = Dict(k => v for (k, v) in column_mapping if k in names(structured_data))
        rename!(structured_data, existing_columns)
        
        # 2. Convertir tipos de datos
        # Convertir fechas (en Julia usamos Date del paquete Dates)
        structured_data.registration_date = [Date(date_str) for date_str in structured_data.registration_date]
        
        # Asegurar tipos numéricos
        structured_data.user_id = Int64.(structured_data.user_id)
        structured_data.total_spent = Float64.(structured_data.total_spent)
        
        # 3. Agregar metadatos de procesamiento
        structured_data.processed_at = fill(now(), nrow(structured_data))
        structured_data.data_source = fill("user_database", nrow(structured_data))
        
        # 4. Reordenar columnas según esquema del data warehouse
        column_order = [:user_id, :user_name, :email_address, :country_code, 
                       :registration_date, :total_spent, :processed_at, :data_source]
        
        # Solo incluir columnas que existen
        available_columns = [col for col in column_order if col in names(structured_data)]
        structured_data = select(structured_data, available_columns)
        
        println("--- 3. Datos Estructurados (Esquema Normalizado) ---")
        println("Columnas finales: ", names(structured_data))
        println(structured_data)
        println("\nInformación del esquema final:")
        for (name, type) in zip(names(structured_data), eltype.(eachcol(structured_data)))
            println("  $name: $type")
        end
        println("")
        
        @info "Normalización completada: $(nrow(structured_data)) registros estructurados"
        return structured_data
        
    catch e
        @error "Error en la normalización: $e"
        rethrow(e)
    end
end

normalize_schema (generic function with 1 method)

### ▶️ Ejecutemos la Normalización

Ahora vamos a normalizar el esquema de nuestros datos limpios:

In [18]:
# Ejecutar la normalización (necesitamos los datos transformados)
if @isdefined(cleaned_data)
    structured_data = normalize_schema(cleaned_data)
else
    println("⚠️ Primero ejecuta las celdas de extracción y transformación")
end

[ Info: Iniciando normalización del esquema...
--- Análisis del Esquema Original ---
Columnas originales: ["ID_USER", "user_name", "registration_date", "total_spent", "country_code", "email"]
Tipos de datos originales:
  ID_USER: Int64
  user_name: String
  registration_date: String
  total_spent: Float64
  country_code: String
  email: String

┌ Error: Error en la normalización: ArgumentError("column name :user_id not found in the data frame")
└ @ Main ~/Documents/ETL-ELT-EAI/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y104sZmlsZQ==.jl:70


ArgumentError: ArgumentError: column name :user_id not found in the data frame

## 4. Load: Carga de Datos

### 🎯 Objetivos de la Fase Load
- Cargar datos en el sistema de destino
- Mantener integridad referencial
- Optimizar el rendimiento de carga
- Implementar estrategias de carga incremental

### 📊 Estrategias de Carga
- **Full Load**: Carga completa de todos los datos
- **Incremental Load**: Solo datos nuevos o modificados
- **Upsert**: Insertar nuevos registros, actualizar existentes
- **SCD (Slowly Changing Dimensions)**: Manejo de cambios históricos

La fase final es cargar los datos transformados en el sistema de destino (Data Warehouse, Data Lake, etc.).

**Ejemplo:** Simularemos diferentes tipos de carga:

In [19]:
function load_to_warehouse(structured_data::DataFrame, load_type="full")
    """
    Carga datos al data warehouse con diferentes estrategias.
    
    Args:
        structured_data (DataFrame): Datos estructurados a cargar
        load_type (String): Tipo de carga ("full", "incremental", "upsert")
    
    Returns:
        Bool: true si la carga fue exitosa
    """
    try
        @info "Iniciando carga de datos - Tipo: $load_type"
        
        # Validaciones pre-carga
        if nrow(structured_data) == 0
            @warn "No hay datos para cargar"
            return false
        end
        
        println("--- Validaciones Pre-Carga ---")
        println("Registros a cargar: ", nrow(structured_data))
        println("Columnas: ", names(structured_data))
        
        # Verificar duplicados
        if "user_id" in names(structured_data)
            duplicates = nrow(structured_data) - length(unique(structured_data.user_id))
            if duplicates > 0
                println("⚠️  Encontrados $duplicates registros duplicados")
                structured_data = unique(structured_data, :user_id)
            end
        end
        
        # Simular diferentes tipos de carga
        filename = ""
        if load_type == "full"
            # Carga completa - reemplaza todos los datos
            filename = "data_warehouse_users_full.csv"
            CSV.write(filename, structured_data)
            println("✅ Carga completa realizada: $filename")
            
        elseif load_type == "incremental"
            # Carga incremental - solo nuevos registros
            filename = "data_warehouse_users_incremental.csv"
            # En un caso real, verificaríamos qué registros ya existen
            CSV.write(filename, structured_data, append=true)
            println("✅ Carga incremental realizada: $filename")
            
        elseif load_type == "upsert"
            # Upsert - insertar nuevos, actualizar existentes
            filename = "data_warehouse_users_upsert.csv"
            CSV.write(filename, structured_data)
            println("✅ Upsert realizado: $filename")
        end
        
        println("\n--- 4. Datos Cargados ---")
        println("Registros cargados exitosamente: ", nrow(structured_data))
        println("\nPrimeras filas del archivo de destino:")
        
        # Mostrar contenido del archivo
        if isfile(filename)
            lines = readlines(filename)
            for (i, line) in enumerate(lines[1:min(6, length(lines))])
                println(line)
            end
        end
        
        # Estadísticas de carga
        println("\n--- Estadísticas de Carga ---")
        println("📊 Total de registros: ", nrow(structured_data))
        if "country_code" in names(structured_data)
            println("📊 Países únicos: ", length(unique(structured_data.country_code)))
        end
        if "total_spent" in names(structured_data)
            println("📊 Gasto promedio: \$", round(mean(structured_data.total_spent), digits=2))
            println("📊 Gasto total: \$", round(sum(structured_data.total_spent), digits=2))
        end
        
        @info "Carga completada exitosamente: $(nrow(structured_data)) registros"
        return true
        
    catch e
        @error "Error en la carga: $e"
        return false
    end
end

load_to_warehouse (generic function with 2 methods)

### ▶️ Ejecutemos la Carga

Ahora vamos a cargar nuestros datos estructurados al data warehouse:

In [20]:
# Ejecutar la carga (necesitamos los datos estructurados)
if @isdefined(structured_data)
    success = load_to_warehouse(structured_data, "full")
    println("\n🎉 Carga completada: $success")
else
    println("⚠️ Primero ejecuta todas las celdas anteriores")
end

⚠️ Primero ejecuta todas las celdas anteriores


## 🔄 Pipeline ETL Completo

Ahora ejecutemos todo el pipeline de una vez:

In [21]:
function run_complete_etl_pipeline()
    """
    Ejecuta el pipeline ETL completo de principio a fin.
    """
    println("🚀 Iniciando Pipeline ETL Completo")
    println("=" ^ 50)
    
    try
        # 1. Extract
        println("\n1️⃣ EXTRACCIÓN")
        raw_data = extract_from_database()
        
        # 2. Transform
        println("\n2️⃣ TRANSFORMACIÓN")
        cleaned_data = apply_business_rules(raw_data)
        
        # 3. Normalize
        println("\n3️⃣ NORMALIZACIÓN")
        structured_data = normalize_schema(cleaned_data)
        
        # 4. Load
        println("\n4️⃣ CARGA")
        success = load_to_warehouse(structured_data, "full")
        
        if success
            println("\n🎉 ¡PIPELINE ETL COMPLETADO EXITOSAMENTE!")
            println("=" ^ 50)
            println("📊 Resumen Final:")
            println("   • Registros procesados: $(nrow(structured_data))")
            println("   • Columnas finales: $(length(names(structured_data)))")
            println("   • Archivo generado: data_warehouse_users_full.csv")
        else
            println("❌ Error en el pipeline")
        end
        
        return structured_data
        
    catch e
        @error "Error en el pipeline ETL: $e"
        return nothing
    end
end

# Ejecutar el pipeline completo
final_data = run_complete_etl_pipeline()

🚀 Iniciando Pipeline ETL Completo

1️⃣ EXTRACCIÓN
[ Info: Iniciando extracción de datos...
--- 1. Datos Crudos Extraídos ---
Registros extraídos: 6
Columnas: ["ID_USER", "user_name", "registration_date", "total_spent", "country_code", "email"]

Primeras filas:
6×6 DataFrame
 Row │ ID_USER  user_name  registration_date  total_spent  country_code  email           
     │ Int64    String?    String             Any          String        String          
─────┼───────────────────────────────────────────────────────────────────────────────────
   1 │     101  Ana        2025-01-15         150.5        ES            ana@email.com
   2 │     102  Luis       2025-02-20         80.0         MX            luis@email.com
   3 │     103  Marta      2025-03-01         -999.0       ES            marta@email.com
   4 │     104  Juan       2025-04-10         200.0        CO            juan@email.com
   5 │     105  Eva        2025-05-19         45.25        es            eva@email.com
   6 │     106  

## 📊 Monitoreo y Calidad de Datos

### 🔍 Validaciones de Calidad

Es crucial monitorear la calidad de los datos en cada etapa:

In [22]:
function data_quality_report(df::DataFrame, stage_name::String)
    """
    Genera un reporte de calidad de datos.
    
    Args:
        df (DataFrame): DataFrame a analizar
        stage_name (String): Nombre de la etapa del pipeline
    """
    println("\n📋 REPORTE DE CALIDAD - $stage_name")
    println("=" ^ 40)
    
    # Estadísticas básicas
    println("📊 Estadísticas Básicas:")
    println("   • Total de registros: $(nrow(df))")
    println("   • Total de columnas: $(ncol(df))")
    println("   • Columnas: $(join(names(df), ", "))")
    
    # Valores faltantes
    println("\n🔍 Valores Faltantes:")
    for col in names(df)
        missing_count = sum(ismissing.(df[!, col]))
        missing_pct = round(missing_count / nrow(df) * 100, digits=2)
        if missing_count > 0
            println("   • $col: $missing_count ($missing_pct%)")
        end
    end
    
    # Duplicados
    if "user_id" in names(df)
        duplicates = nrow(df) - length(unique(df.user_id))
        println("\n🔄 Duplicados:")
        println("   • Registros duplicados: $duplicates")
    end
    
    # Validaciones específicas por columna
    println("\n✅ Validaciones Específicas:")
    
    if "email" in names(df) || "email_address" in names(df)
        email_col = "email" in names(df) ? "email" : "email_address"
        valid_emails = sum(occursin.(r"@", df[!, email_col]))
        println("   • Emails válidos: $valid_emails/$(nrow(df))")
    end
    
    if "age" in names(df)
        valid_ages = sum((df.age .>= 0) .& (df.age .<= 120))
        println("   • Edades válidas (0-120): $valid_ages/$(nrow(df))")
    end
    
    if "total_spent" in names(df)
        valid_amounts = sum(df.total_spent .>= 0)
        println("   • Montos válidos (>=0): $valid_amounts/$(nrow(df))")
    end
    
    println("\n" * "=" ^ 40)
end

# Generar reportes de calidad para cada etapa
if @isdefined(raw_data)
    data_quality_report(raw_data, "EXTRACCIÓN")
end

if @isdefined(cleaned_data)
    data_quality_report(cleaned_data, "TRANSFORMACIÓN")
end

if @isdefined(structured_data)
    data_quality_report(structured_data, "NORMALIZACIÓN")
end


📋 REPORTE DE CALIDAD - EXTRACCIÓN
📊 Estadísticas Básicas:
   • Total de registros: 6
   • Total de columnas: 6
   • Columnas: ID_USER, user_name, registration_date, total_spent, country_code, email

🔍 Valores Faltantes:
   • user_name: 1 (16.67%)

✅ Validaciones Específicas:
   • Emails válidos: 6/6


MethodError: MethodError: no method matching isless(::Int64, ::String)
The function `isless` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  isless(!Matched::Missing, ::Any)
   @ Base missing.jl:87
  isless(::Any, !Matched::Missing)
   @ Base missing.jl:88
  isless(::Real, !Matched::AbstractFloat)
   @ Base operators.jl:179
  ...


## 🎯 Mejores Prácticas ETL en Julia

### 📋 Checklist de Mejores Prácticas

#### ✅ Diseño del Pipeline
- **Modularidad**: Cada función tiene una responsabilidad específica
- **Reutilización**: Funciones genéricas que se pueden usar en múltiples pipelines
- **Configurabilidad**: Parámetros externos para diferentes entornos

#### ✅ Manejo de Errores
- **Try-catch**: Captura y manejo de excepciones
- **Logging**: Uso de `@info`, `@warn`, `@error` para trazabilidad
- **Validaciones**: Verificación de datos en cada etapa

#### ✅ Performance
- **Broadcasting**: Uso de `.` para operaciones vectorizadas
- **Memory efficiency**: Evitar copias innecesarias de DataFrames
- **Parallel processing**: Usar `@threads` para operaciones paralelas

#### ✅ Calidad de Datos
- **Validaciones**: Verificar integridad en cada paso
- **Monitoreo**: Reportes de calidad automáticos
- **Documentación**: Funciones bien documentadas

### 🔧 Herramientas Complementarias

#### Paquetes Julia para ETL:
- **DataFrames.jl**: Manipulación de datos tabulares
- **CSV.jl**: Lectura/escritura de archivos CSV
- **JSON3.jl**: Manejo de datos JSON
- **HTTP.jl**: Consumo de APIs REST
- **Dates.jl**: Manipulación de fechas
- **Statistics.jl**: Estadísticas descriptivas
- **Query.jl**: Consultas tipo LINQ
- **MLJ.jl**: Machine Learning (para pipelines ML)

#### Bases de Datos:
- **LibPQ.jl**: PostgreSQL
- **MySQL.jl**: MySQL
- **SQLite.jl**: SQLite
- **ODBC.jl**: Conexiones ODBC genéricas

## 🏋️ Ejercicios Prácticos

### Ejercicio 1: Pipeline de Ventas
Crea un pipeline ETL para procesar datos de ventas con las siguientes características:
- Extraer datos de múltiples archivos CSV
- Calcular métricas de ventas por región
- Detectar transacciones anómalas
- Generar reporte consolidado

### Ejercicio 2: Pipeline de Logs
Desarrolla un pipeline para procesar logs de aplicación:
- Parsear logs en formato no estructurado
- Extraer información de errores
- Calcular métricas de performance
- Alertas automáticas

### Ejercicio 3: Pipeline de Redes Sociales
Construye un pipeline para analizar datos de redes sociales:
- Consumir API de Twitter/Facebook
- Análisis de sentimientos
- Detección de tendencias
- Dashboard en tiempo real

### 💡 Desafío Avanzado
Implementa un pipeline ETL que:
1. Procese datos en tiempo real usando streaming
2. Implemente machine learning para detección de anomalías
3. Use paralelización para mejorar performance
4. Incluya tests automatizados
5. Tenga monitoreo y alertas

## 📚 Recursos Adicionales

### 📖 Documentación Oficial
- [Julia Documentation](https://docs.julialang.org/)
- [DataFrames.jl Guide](https://dataframes.juliadata.org/stable/)
- [CSV.jl Documentation](https://csv.juliadata.org/stable/)

### 🎓 Cursos y Tutoriales
- [Julia Academy](https://juliaacademy.com/)
- [Think Julia Book](https://benlauwens.github.io/ThinkJulia.jl/latest/book.html)
- [Julia Data Science](https://juliadatascience.io/)

### 🛠️ Herramientas y Frameworks
- [Pluto.jl](https://github.com/fonsp/Pluto.jl) - Notebooks reactivos
- [Genie.jl](https://genieframework.com/) - Framework web
- [MLJ.jl](https://alan-turing-institute.github.io/MLJ.jl/dev/) - Machine Learning
- [Flux.jl](https://fluxml.ai/) - Deep Learning

### 🌐 Comunidad
- [Julia Discourse](https://discourse.julialang.org/)
- [Julia Slack](https://julialang.org/slack/)
- [JuliaCon](https://juliacon.org/) - Conferencia anual

### 📊 Casos de Uso Reales
- **Finanzas**: Análisis de riesgo, trading algorítmico
- **Ciencia**: Simulaciones científicas, bioinformática
- **ML/AI**: Modelos de machine learning, deep learning
- **Data Engineering**: Pipelines ETL, procesamiento distribuido

---

## 🎉 ¡Felicitaciones!

Has completado el tutorial de ETL con Julia. Ahora tienes las herramientas para:

✅ Construir pipelines ETL robustos y escalables  
✅ Manejar datos de diferentes fuentes y formatos  
✅ Implementar validaciones y monitoreo de calidad  
✅ Aplicar mejores prácticas de ingeniería de datos  
✅ Usar las características únicas de Julia para performance  

**¡Sigue practicando y construyendo pipelines cada vez más sofisticados!** 🚀